# **Stock Prices Predictions with DeepAR**

This Notebook will contain the modeling phases needed to predict stock prices using a deep learning model.
The stocks analyzed will be the following:
* IBM
* AAPL (Apple Inc.)
* AMZN (Amazon Inc.)
* GOOGL (Alphabet Inc.)


In [226]:
import sagemaker

# Data preparation

Data must be prepared in order to be processed by DeepAR model:
* Train/test set split
* Save Data locally
* Upload to S3

In [227]:
import os

In [228]:
data_dir = 'stock_deepar'

In [229]:
# The folder we will be used to store json data
data_dir_json = os.path.join(data_dir, 'json')

In [230]:
# The folder we will be used to store csv data
data_dir_csv = os.path.join(data_dir, 'csv')

In [231]:
# The folder we will be used to store training data in json format
data_dir_json_train = os.path.join(data_dir_json, 'train')
data_dir_json_train_ibm = os.path.join(data_dir_json_train, 'IBM')
data_dir_json_train_aapl = os.path.join(data_dir_json_train, 'AAPL')
data_dir_json_train_amzn = os.path.join(data_dir_json_train, 'AMZN')
data_dir_json_train_googl = os.path.join(data_dir_json_train, 'GOOGL')

In [232]:
# The folder we will be used to store test data in json format
data_dir_json_test = os.path.join(data_dir_json, 'test')
data_dir_json_test_ibm = os.path.join(data_dir_json_test, 'IBM')
data_dir_json_test_aapl = os.path.join(data_dir_json_test, 'AAPL')
data_dir_json_test_amzn = os.path.join(data_dir_json_test, 'AMZN')
data_dir_json_test_googl = os.path.join(data_dir_json_test, 'GOOGL')

In [233]:
# The folder we will be used to store validation data in json format
data_dir_json_valid = os.path.join(data_dir_json, 'validation')
data_dir_json_valid_ibm = os.path.join(data_dir_json_valid, 'IBM')
data_dir_json_valid_aapl = os.path.join(data_dir_json_valid, 'AAPL')
data_dir_json_valid_amzn = os.path.join(data_dir_json_valid, 'AMZN')
data_dir_json_valid_googl = os.path.join(data_dir_json_valid, 'GOOGL')

In [234]:
# The folder we will be used to store moving average benchmark model output data in json format
data_dir_json_benchmark = os.path.join(data_dir_json, 'benchmark')

In [243]:
# initializing train/test dataframe lists to iterate on them
dfs_train = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]
dfs_test = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]
dfs_valid = [df_ibm_valid, df_aapl_valid, df_amzn_valid, df_googl_valid]

## Save Data Locally

In [244]:
if not os.path.exists(data_dir_csv): # Make sure that the folder exists
    os.makedirs(data_dir_csv)

In [245]:
# IBM
df_ibm_train.to_csv(os.path.join(data_dir_csv, 'ibm_train.csv'), header=True, index=True)
df_ibm_test.to_csv(os.path.join(data_dir_csv, 'ibm_test.csv'), header=True, index=True)
df_ibm_valid.to_csv(os.path.join(data_dir_csv, 'ibm_valid.csv'), header=True, index=True)

In [246]:
# Apple Inc.
df_aapl_train.to_csv(os.path.join(data_dir_csv, 'aapl_train.csv'), header=True, index=True)
df_aapl_test.to_csv(os.path.join(data_dir_csv, 'aapl_test.csv'), header=True, index=True)
df_aapl_valid.to_csv(os.path.join(data_dir_csv, 'aapl_valid.csv'), header=True, index=True)

In [247]:
# Amazon.com
df_amzn_train.to_csv(os.path.join(data_dir_csv, 'amzn_train.csv'), header=True, index=True)
df_amzn_test.to_csv(os.path.join(data_dir_csv, 'amzn_test.csv'), header=True, index=True)
df_amzn_valid.to_csv(os.path.join(data_dir_csv, 'amzn_valid.csv'), header=True, index=True)

In [248]:
# Alphabet Inc.
df_googl_train.to_csv(os.path.join(data_dir_csv, 'googl_train.csv'), header=True, index=True)
df_googl_test.to_csv(os.path.join(data_dir_csv, 'googl_test.csv'), header=True, index=True)
df_googl_valid.to_csv(os.path.join(data_dir_csv, 'googl_valid.csv'), header=True, index=True)

### JSON serialization

In order to feed DeepAR model, JSON files must be prepared from data.
I'll dispose two kind of JSON inputs:
* one with "dynamic features", to use a DeepAR API terminology: all dataset features except for target column and related one ('Adj Close', 'Close');
* one without "dynamic features: only 'Adj Close' column will be fed to DeepAR model.

#### DataFrame to JSON conversion

Now I'm going to convert data to JSON file format, in order to feed the DeepAR model correctly

As already announced, I will create two kind of time series, one with a list of dynamic features `dyn_feat`and the other one with only the target column (`Adj Close`) time series. 

Creating local storage path:

In [249]:
if not os.path.exists(data_dir_json): # Make sure that the folder exists
    os.makedirs(data_dir_json)

Serializing data to json files

In [250]:
from source_deepar.deepar_utils import ts2dar_json

Dataset with the `Adj Close` time series alone:

### Training data:

In [251]:
if not os.path.exists(data_dir_json_train): # Make sure that the folder exists
    os.makedirs(data_dir_json_train)

In [252]:
for df, m in zip(dfs_train, mnemonics):
    ts2dar_json(df, data_dir_json_train, m+'.json')

### Test data:

In [253]:
if not os.path.exists(data_dir_json_test): # Make sure that the folder exists
    os.makedirs(data_dir_json_test)

In [254]:
for df, m in zip(dfs_test, mnemonics):
    ts2dar_json(df, data_dir_json_test, m+'.json')

### Validation data:

In [261]:
if not os.path.exists(data_dir_json_valid): # Make sure that the folder exists
    os.makedirs(data_dir_json_valid)

In [262]:
for df, m in zip(dfs_valid, mnemonics):
    ts2dar_json(df, data_dir_json_valid, m+'.json')

### Benchmark data:

In [258]:
if not os.path.exists(data_dir_json_benchmark): # Make sure that the folder exists
    os.makedirs(data_dir_json_benchmark)

In [259]:
for s, m in zip(ser_valid_ma, mnemonics):
    ts2dar_json(df, data_dir_json_benchmark, m+'.json')

## AWS declarations

Defining training data Location

In [260]:
# Define IAM role and session
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

In [261]:
interval ='D'

Define S3 variables for model train artifacts:

In [262]:
#Define training data location
s3_data_key = 'train_artifacts'
s3_bucket = sagemaker_session.default_bucket()
s3_output_path = "s3://{}/{}/{}/{}/output".format(s3_bucket, data_dir, s3_data_key, interval)

In [263]:
#Obtain container image URI for SageMaker-DeepAR algorithm, based on region
region = sagemaker_session.boto_region_name
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)
print("Model image : {}".format(image_name))

Model image : 495149712605.dkr.ecr.eu-central-1.amazonaws.com/forecasting-deepar:1


## Upload data to S3

Training input preparation, all data togheter:

In [268]:
# *unique* train/test prefixes
train_prefix   = '{}/{}'.format(data_dir_json, 'train')
test_prefix    = '{}/{}'.format(data_dir_json, 'test')
valid_prefix = '{}/{}'.format(data_dir_json, 'valid')
benchmark_prefix = '{}/{}'.format(data_dir_json, 'benchmark')

In [269]:
input_data_train = sagemaker_session.upload_data(path=data_dir_json_train, bucket=s3_bucket, key_prefix=train_prefix)

In [270]:
input_data_test = sagemaker_session.upload_data(path=data_dir_json_test, bucket=s3_bucket, key_prefix=test_prefix)

In [271]:
data_valid = sagemaker_session.upload_data(path=data_dir_json_valid, bucket=s3_bucket, key_prefix=valid_prefix)

In [273]:
benchmark_valid = sagemaker_session.upload_data(path=data_dir_json_benchmark, bucket=s3_bucket, key_prefix=benchmark_prefix)

### Set DeepAR specific hyperparameters

In [274]:
# setting target columns
target_column = 'Adj Close'

In [275]:
# DeepAR estimator parameters    
hyperparameters = {
    "prediction_length": str(prediction_length[1]), #number of time-steps model is trained to predict, always generates forecasts with this length
    "context_length": str(context_length[1]), #number of time-points that the model gets to see before making the prediction, should be about same as the prediction_length
    "time_freq": interval, #granularity of the time series in the dataset
    "epochs": "300", #maximum number of passes over the training data
    "early_stopping_patience": "40", #training stops when no progress is made within the specified number of epochs
    "num_layers": "4", #number of hidden layers in the RNN, typically range from 1 to 4    
    "num_cells": "40", #number of cells to use in each hidden layer of the RNN, typically range from 30 to 100
    "mini_batch_size": "128", #size of mini-batches used during training, typically values range from 32 to 512
    "learning_rate": "1e-3", #learning rate used in training. Typical values range from 1e-4 to 1e-1
    "dropout_rate": "0.12", # dropout rate to use for regularization, typically less than 0.2. 
    "likelihood": "student-T" #noise model used for uncertainty estimates - gaussian/beta/negative-binomial/student-T/deterministic-L1
}

## Estimator Instantiation

In [276]:
from sagemaker.estimator import Estimator

Estimator to be used with all data togheter:

In [277]:
# instantiate a DeepAR estimator
estimator = Estimator(image_uri=image_name,
                      sagemaker_session=sagemaker_session,
                      #image_name=image_name,
                      role=role,
                      instance_count=1,
                      instance_type='ml.c4.xlarge',
                      output_path=s3_output_path,
                      hyperparameters=hyperparameters
                      )

## Training Job Creation

Creation of a training job with stand alone time series (no dynamic features provided). Run only if no model has already been trained before.

fit estimator on all data together:

In [278]:
from datetime import date

In [279]:
today = date.today()

In [280]:
# increment each time you don't run the training, to keep pointing to the right training artifact package

In [281]:
n = 2

In [282]:
n_daysago = today - datetime.timedelta(n)

In [283]:
all_data_training_job_name = 'deepar-All-{}'.format(n_daysago)

# SKIP if trainining the model is not needed!

In [284]:
# train and test channels
data_channels = {
    "train": input_data_train,
    "test": input_data_test
}

In [ ]:
%%time
estimator.fit(inputs=data_channels, job_name=all_data_training_job_name, logs='None')

## Existing Model Instantiation

Instatiation of a model from existing training artifacts (run only if a model has already been trained before).

Model trained on all data together:

In [285]:
all_stocks_model = sagemaker.model.Model(
    model_data='{}/{}/output/model.tar.gz'.format(s3_output_path, all_data_training_job_name),
    image_uri= image_name,
    role=role)  # your role here; could be different name

#trainedmodel.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge')

## Deploy and Create a Predictor

Now that we have trained a model, we can use it to perform predictions by deploying it to a predictor endpoint.

Remember to **delete the endpoint** at the end of this notebook. A cell at the very bottom of this notebook will be provided, but it is always good to keep, front-of-mind.

In [286]:
# importing serializers
from sagemaker.predictor import json_serializer, json_deserializer

Deploying endpoint for global (all stocks together) estimator:

In [287]:
all_data_endpoint_name = 'DeepAR-ml-spp'

#### run it once, then update the endpoint if needed

In [288]:
%%time
all_data_endpoint = all_stocks_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name=all_data_endpoint_name,
    serializer=json_serializer,
    deserializer=json_deserializer
)

-----------------------!CPU times: user 428 ms, sys: 27.1 ms, total: 455 ms
Wall time: 11min 32s


#### update endpoint if needed:

All data model:

In [ ]:
%%time
endpoint = all_data_endpoint.update_endpoint(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
)

# Generating Predictions

According to the [inference format](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar-in-formats.html) for DeepAR, the `predictor` expects to see input data in a JSON format, with the following keys:
* **instances**: A list of JSON-formatted time series that should be forecast by the model.
* **configuration** (optional): A dictionary of configuration information for the type of response desired by the request.

Within configuration the following keys can be configured:
* **num_samples**: An integer specifying the number of samples that the model generates when making a probabilistic prediction.
* **output_types**: A list specifying the type of response. We'll ask for **quantiles**, which look at the list of num_samples generated by the model, and generate [quantile estimates](https://en.wikipedia.org/wiki/Quantile) for each time point based on these values.
* **quantiles**: A list that specified which quantiles estimates are generated and returned in the response.


Below is an example of what a JSON query to a DeepAR model endpoint might look like.

```
{
 "instances": [
  { "start": "2009-11-01 00:00:00", "target": [4.0, 10.0, 50.0, 100.0, 113.0] },
  { "start": "1999-01-30", "target": [2.0, 1.0] }
 ],
 "configuration": {
  "num_samples": 50,
  "output_types": ["quantiles"],
  "quantiles": ["0.5", "0.9"]
 }
}
```

## Instantiate a Predictor

Instantiate a predictor to preprocess input data for predictions

In [ ]:
from source_deepar.deepar_utils import DeepARPredictor

All data based predictor:

In [ ]:
all_data_predictor = DeepARPredictor(endpoint_name=all_data_endpoint_name, sagemaker_session=sagemaker_session)
all_data_predictor.set_prediction_parameters(interval, prediction_length[1])

## Get Predictions

We can now use the model to get a predictions for input time series.

### Predictions on test dataset

Ground truth:

In [ ]:
test_gt = [df_ibm_test.iloc[-prediction_length[1]:]['Adj Close'], df_aapl_test.iloc[-prediction_length[1]:]['Adj Close'],
           df_amzn_test.iloc[-prediction_length[1]:]['Adj Close'], df_googl_test.iloc[-prediction_length[1]:]['Adj Close']]

Prediction input:

In [ ]:
# get all input and target (test) time series
input_ts = [df_ibm_train, df_aapl_train, df_amzn_train, df_googl_train]

Prediction output on all data together:

In [ ]:
# get predictions for all the time series
test_predictions = all_data_predictor.predict(input_ts)

Take a look at one of them:

As we can see, index are just progressing of one day each row, wich is not stock price progression scheme in real life (e.g.: weekends are not trading days), so I'm going fix the index before going on with the analysis of results:

All data together:

In [ ]:
for i in range(0, len(test_predictions)):
    test_predictions[i].index = test_gt[i].index

Save data locally:

In [ ]:
data_dir_json_prediction = os.path.join(data_dir_json, 'prediction') # The folder we will use for storing data
if not os.path.exists(data_dir_json_prediction): # Make sure that the folder exists
    os.makedirs(data_dir_json_prediction)

All data together:

In [ ]:
# since all of the predictions start and end on the same day, i just use one of them to get the datetime string
start_date = test_predictions[0].index[0].date().strftime(format="%Y-%m-%d")
end_date = test_predictions[0].index[-1].date().strftime(format="%Y-%m-%d")

Prediction serialization:

All data together:

In [ ]:
for i in range(0, len(test_predictions)):
    test_predictions[i].to_json(os.path.join(data_dir_json_prediction,
                                             "{}_{} - {}.json".format(mnemonics[i], start_date, end_date)),
                                orient='columns',date_format='iso')

Prediction de-serialization:

In [ ]:
dsz_test_predictions = []
for i in range(0, len(test_predictions)):
    dsz_test_predictions.append(
        pd.read_json(os.path.join(data_dir_json_prediction,
                                  "{}_{} - {}.json".format(mnemonics[i], start_date, end_date)),
                     orient='columns', convert_axes=False))

Again, index normalization using target index, before using deserialized data:

In [ ]:
for i in range(0, len(test_predictions)):
    dsz_test_predictions[i].index = test_gt[i].index

## Predicting the Future -  validation data predictions

Now that we've tested our estimator on test set, we would like to see how it behaves on new data.
So we'll feed it with validation data we set apart before starting the training phase.
Create a formatted input to send to the deployed `endpoint` passing usual parameters for "configuration". The "instances" will, in this case, just be one instance, defined by the following:
* **start**: The start time from wich we would like to make a prediction.
* **target**: The target will be an empty list because this time period has no, complete associated time series.
```
{"start": start_time, "target": []} # empty target
```

Creating an array comprising all of the validation dataframes

In [ ]:
df_valid = [df_ibm_valid, df_aapl_valid, df_amzn_valid, df_googl_valid]

In [ ]:
df_test = [df_ibm_test, df_aapl_test, df_amzn_test, df_googl_test]

Retrieving ground truth data:

In [ ]:
valid_gt = [data['Adj Close'] for data in df_valid]

Set start date as validation set start date, in order to complete the benchmark metrics task, I need to compare the performance of the model on the same dataset.

All data together:

In [ ]:
# formatting start_date of each of the validation dataset (should be the same)
start_date = [data.index[0] for data in df_valid]

Retrieving predictions for all data together:

In [ ]:
# get prediction response on validation data
valid_predictions = all_data_predictor.predict_future(start_date)

In [ ]:
valid_predictions = all_data_predictor.predict(df_test)

Retrieving predictions for stand alone stocks:

In [ ]:
print(start_date)

Normalizing index to real world trading dates:

All data together:

In [ ]:
for i in range(0, len(valid_predictions)):
    valid_predictions[i].index = valid_gt[i].index

Save data locally:

All data together:

In [ ]:
# since all of the predictions start and end on the same day, i just use one of them to get the datetime string
start_date = valid_predictions[0].index[0].date().strftime(format="%Y-%m-%d")
end_date = valid_predictions[0].index[-1].date().strftime(format="%Y-%m-%d")

Prediction serialization:

All data together:

In [ ]:
for i in range(0, len(valid_predictions)):
    valid_predictions[i].to_json(os.path.join(data_dir_json_prediction,
                                             "{}_{} - {}_valid.json".format(mnemonics[i], start_date, end_date)),
                                orient='columns',date_format='iso')

Prediction de-serialization:

In [ ]:
dsz_valid_predictions = []
for i in range(0, len(valid_predictions)):
    dsz_valid_predictions.append(
        pd.read_json(os.path.join(data_dir_json_prediction,
                                  "{}_{} - {}_valid.json".format(mnemonics[i], start_date, end_date)),
                     orient='columns', convert_axes=False))

Again, index normalization using target index, before using deserialized data:

In [ ]:
for i in range(0, len(valid_predictions)):
    dsz_valid_predictions[i].index = valid_gt[i].index

## Results visualization

The quantile data will give us all we need to see the results of our prediction.
* Quantiles 0.1 and 0.9 represent higher and lower bounds for the predicted values.
* Quantile 0.5 represents the median of all sample predictions.

In [ ]:
from source_deepar.display_quantiles import display_quantiles

Visualization of predictions on test data

All together data:

In [ ]:
# display predictions
for i in range(len(test_predictions)):
    display_quantiles(dsz_valid_predictions[i], valid_gt[i], ser_valid_ma[i], 'moving average')

In [ ]:
from source_deepar.display_quantiles import display_quantiles_flask

Visualization of predictions on test data

All together data:

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
# display predictions
quantiles_img = []
for i in range(len(test_predictions)):
    quantiles_img.append(display_quantiles_flask(dsz_valid_predictions[i], valid_gt[i], ser_valid_ma[i], 'moving average'))

In [ ]:
display(quantiles_img[0])

Stand alone predicted stocks:

In [ ]:
dsz_valid_predictions[3]

In [ ]:
# display predictions
for i in range(len(dsz_valid_predictions)):
    display_quantiles(dsz_valid_predictions[i], valid_gt[i], ser_valid_ma[i], 'moving average')

## Metrics computation

Now that we have predictions on validation dataset, we can compute our metrics and compare it to benchmakr model performaces.

### IBM Stock prices

Mean Absolute Error

In [ ]:
ibm_dar_mae_loss = mean_absolute_error(test_gt[0], dsz_valid_predictions[0]['0.5'])

In [ ]:
print(ibm_dar_mae_loss)

Root Mean Squared Error

In [ ]:
ibm_dar_mse_loss = mean_squared_error(test_gt[0], dsz_valid_predictions[0]['0.5'], squared=False)

In [ ]:
print(ibm_dar_mse_loss)

Mean Absolute Percentage Error

In [ ]:
ibm_dar_map_loss = mean_absolute_percentage_error(test_gt[0], dsz_valid_predictions[0]['0.5'])

In [ ]:
print(ibm_dar_map_loss)

R<sup>2</sup> score

In [ ]:
ibm_dar_r2_score = r2_score(test_gt[0], dsz_valid_predictions[0]['0.5'])

In [ ]:
print(ibm_dar_r2_score)

### AAPL Stock prices

Mean Absolute Error

In [ ]:
aapl_dar_mae_loss = mean_absolute_error(test_gt[1], dsz_valid_predictions[1]['0.5'])

In [ ]:
print(aapl_dar_mae_loss)

Root Mean Squared Error

In [ ]:
aapl_dar_mse_loss = mean_squared_error(test_gt[1], dsz_valid_predictions[1]['0.5'], squared=False)

In [ ]:
print(aapl_dar_mse_loss)

Mean Absolute Percentage Error

In [ ]:
aapl_dar_map_loss = mean_absolute_percentage_error(test_gt[1], dsz_valid_predictions[1]['0.5'])

In [ ]:
print(aapl_dar_map_loss)

R<sup>2</sup> score

In [ ]:
aapl_dar_r2_score = r2_score(test_gt[1], dsz_valid_predictions[1]['0.5'])

In [ ]:
print(aapl_dar_r2_score)

### AMZN Stock prices

Mean Absolute Error

In [ ]:
amzn_dar_mae_loss = mean_absolute_error(test_gt[2], dsz_valid_predictions[2]['0.5'])

In [ ]:
print(amzn_dar_mae_loss)

Root Mean Squared Error

In [ ]:
amzn_dar_mse_loss = mean_squared_error(test_gt[2], dsz_valid_predictions[2]['0.5'], squared=False)

In [ ]:
print(amzn_dar_mse_loss)

Mean Absolute Percentage Error

In [ ]:
amzn_dar_map_loss = mean_absolute_percentage_error(test_gt[2], dsz_valid_predictions[2]['0.5'])

In [ ]:
print(amzn_dar_map_loss)

R<sup>2</sup> score

In [ ]:
amzn_dar_r2_score = r2_score(test_gt[2], dsz_valid_predictions[2]['0.5'])

In [ ]:
print(amzn_dar_r2_score)

### GOOGL Stock prices

Mean Absolute Error

In [ ]:
googl_dar_mae_loss = mean_absolute_error(test_gt[3], dsz_valid_predictions[3]['0.5'])

In [ ]:
print(googl_dar_mae_loss)

Root Mean Squared Error

In [ ]:
googl_dar_mse_loss = mean_squared_error(test_gt[3], dsz_valid_predictions[3]['0.5'], squared=False)

In [ ]:
print(googl_dar_mse_loss)

Mean Absolute Percentage Error

In [ ]:
googl_dar_map_loss = mean_absolute_percentage_error(test_gt[3], dsz_valid_predictions[3]['0.5'])

In [ ]:
print(googl_dar_map_loss)

R<sup>2</sup> score

In [ ]:
googl_dar_r2_score = r2_score(test_gt[3], dsz_valid_predictions[3]['0.5'])

In [ ]:
print(googl_dar_r2_score)

## Conclusions

With reference to all the metrics, the DeepAR method has performed worse than the simple moving average in Adjusted Close stock prices predictions. Maybe better performances could be achieved by means of additional feature analyis.

## Delete the Endpoint

Try your code out on different time series. You may want to tweak your DeepAR hyperparameters and see if you can improve the performance of this predictor.

When you're done with evaluating the predictor (any predictor), make sure to delete the endpoint.

In [ ]:
## TODO: delete the endpoint
all_data_predictor.delete_endpoint()